In [1]:
import pandas as pd

In [2]:
df_recipes = pd.read_csv('../../Datasets/RAW_recipes.csv')
df_interactions = pd.read_csv('../../Datasets/RAW_interactions.csv')

In [3]:
df_grouped = df_interactions.groupby('recipe_id', group_keys=True)
df_ratings = df_grouped.agg({
    'recipe_id':'last',
    'rating':'mean'
})
df_ratings = df_ratings.reset_index(drop=True)
df_ratings = df_ratings.rename(columns={"recipe_id":"id", "rating":"score"})
df_merged = pd.merge(df_recipes, df_ratings, on='id')
df_merged['difficulty'] = df_merged.n_steps * df_merged.minutes
bins = [df_merged['difficulty'].min(), 150, 350, 750, df_merged['difficulty'].max()]
labels = ['easy', 'medium', 'hard', 'very hard']
df_merged['difficulty'] = pd.cut(df_merged['difficulty'], bins=bins, labels=labels)
df_merged = df_merged.drop(df_merged.index[[3381]]) #Error in line formatting

In [4]:
from cassandra.cluster import Cluster

cluster = Cluster(['127.0.0.1'])
session = cluster.connect()

In [5]:
import re
from cassandra.concurrent import execute_concurrent

set_mapping_table = str.maketrans({'[': '{', ']': '}'})
description_mapping_table = str.maketrans({"'": ""})
apostrophe_mapping_table = str.maketrans({"'": "''"})

query_insert_recipes_by_month_submitted = session.prepare("INSERT INTO recipe.recipes_by_month_submitted (month_submitted,score,id,name) VALUES (?,?,?,?)")
query_insert_recipes_by_difficulty = session.prepare("INSERT INTO recipe.recipes_by_difficulty (difficulty,score,id,name) VALUES (?,?,?,?)")
query_insert_recipes = session.prepare("INSERT INTO recipe.recipes (name,tags,date_submitted,score,id,minutes,contributor_id,nutrition,steps,number_of_steps,description,ingredients,number_of_ingredients,difficulty) VALUES (?,?,?,?,?,?,?,?,?,?,?,?,?,?)")
query_insert_recipes_by_tag = session.prepare("INSERT INTO recipe.recipes_by_tag (tag,date_submitted,id,score,name) VALUES (?,?,?,?,?)")
#insert into recipes_by_tag_by_score not needed since it is a materialized view, so it has the data from recipes_by_score

data_insert_recipes_by_month_submitted = []
data_insert_recipes_by_difficulty = []
data_insert_recipes = []
data_insert_recipes_by_tag = []

def convertToListOfFloat(text):
    result = []
    text = text.strip('[' + ']')
    for kv in re.split(',|\*|\n', text):
        kv = kv.strip()
        if(len(kv) == 0):
            continue
        result.append(float(kv))
    return result

def convertToSetOrListOfText(text, typeSelector):
    result = []
    if typeSelector == 'set':
        beginning = '{'
        end = '}'
        text = text.translate(set_mapping_table)
    else:
        beginning = '['
        end = ']'
    text = text.strip(beginning + end)
    for kv in re.split('\', |\", |\*|\n', text):
        kv = kv.strip()
        if(len(kv) == 0):
            continue
        elif(kv[0] == '\"'):
            kv = kv.strip("\"")
        elif(kv[0] == '\''):
            kv = kv.strip("\'")
        kv = kv.translate(apostrophe_mapping_table)
        if(result != ''):
            result.append(kv)
    return result
 
i = 0;
for index, row in df_merged.iterrows():
        tags = convertToSetOrListOfText(str(row['tags']), 'set')
        ingredients = convertToSetOrListOfText(str(row['ingredients']), 'set')
        steps = convertToSetOrListOfText(str(row['steps']), 'list')
        nutrition = convertToListOfFloat(str(row['nutrition']))
        data_insert_recipes_by_month_submitted.append((query_insert_recipes_by_month_submitted, (str(row['submitted'][:-3]),row['score'],row['id'],str(row['name']))))
        data_insert_recipes_by_difficulty.append((query_insert_recipes_by_difficulty, (str(row['difficulty']),row['score'],row['id'],str(row['name']))))
        data_insert_recipes.append((query_insert_recipes, (str(row['name']),tags,str(row['submitted']),row['score'],row['id'],row['minutes'],row['contributor_id'],nutrition,steps,row['n_steps'],str(row['description']).translate(description_mapping_table),ingredients,row['n_ingredients'],str(row['difficulty']))))
        for tag in tags:
            if(tag != ''):
                data_insert_recipes_by_tag.append((query_insert_recipes_by_tag, (tag,str(row['submitted']),row['id'],row['score'],str(row['name']))))

In [6]:
import time

def split(a, n):
    k, m = divmod(len(a), n)
    return (a[i*k+min(i, m):(i+1)*k+min(i+1, m)] for i in range(n))

starting_time = time.time()
    
print("Inserting into recipe.recipes_by_month_submitted")
execute_concurrent(session, data_insert_recipes_by_month_submitted, raise_on_first_error=True)

print("Inserting into recipe.recipes_by_difficulty")
execute_concurrent(session, data_insert_recipes_by_difficulty, raise_on_first_error=True)

print("Inserting into recipe.recipes")
execute_concurrent(session, data_insert_recipes, raise_on_first_error=True)

#Splitting in order to reduce RAM consumption since execute_concurrent return values are garbage collected
i = 1
for data in list(split(data_insert_recipes_by_tag, 10)):
    print(f"Inserting into recipe.recipes_by_tag_{i}")
    execute_concurrent(session, data, raise_on_first_error=True)
    i+=1

elapsed_time = time.time() - starting_time
print('Execution time:', time.strftime("%H:%M:%S", time.gmtime(elapsed_time)))

Inserting into recipe.recipes_by_month_submitted
Inserting into recipe.recipes_by_difficulty
Inserting into recipe.recipes
Inserting into recipe.recipes_by_tag_1
Inserting into recipe.recipes_by_tag_2
Inserting into recipe.recipes_by_tag_3
Inserting into recipe.recipes_by_tag_4
Inserting into recipe.recipes_by_tag_5
Inserting into recipe.recipes_by_tag_6
Inserting into recipe.recipes_by_tag_7
Inserting into recipe.recipes_by_tag_8
Inserting into recipe.recipes_by_tag_9
Inserting into recipe.recipes_by_tag_10
Execution time: 00:16:42


In [8]:
from cassandra.query import SimpleStatement
import time

#Query 1
starting_time = time.time()

query = session.prepare("SELECT name, month_submitted, score FROM recipe.recipes_by_month_submitted  WHERE month_submitted = ? ORDER BY score DESC LIMIT 30;",)

rows = session.execute(query, ['2012-05'])
i = 0;
for row in rows:
    if i < 20:
        print(f"Name: {row.name}, Month Submitted: {row.month_submitted}, Score: {row.score}");
    i+=1

rows = session.execute(query, ['2012-04'])
for row in rows:
    continue;
rows = session.execute(query, ['2012-03'])
for row in rows:
    continue;
rows = session.execute(query, ['2012-02'])
for row in rows:
    continue;
rows = session.execute(query, ['2012-01'])
for row in rows:
    continue;
rows = session.execute(query, ['2011-12'])
for row in rows:
    continue;
rows = session.execute(query, ['2011-11'])
for row in rows:
    continue;
rows = session.execute(query, ['2011-10'])
for row in rows:
    continue;
rows = session.execute(query, ['2011-09'])
for row in rows:
    continue;
rows = session.execute(query, ['2011-08'])
for row in rows:
    continue;

elapsed_time = (time.time() - starting_time) / 10
mlsec = repr(elapsed_time).split('.')[1][:6]
print('Mean Execution time:', time.strftime("%H:%M:%S.{}".format(mlsec), time.gmtime(elapsed_time)))

Name: vegetarian lettuce wraps, Month Submitted: 2012-05, Score: 5.0
Name: nasi goreng  indonesian fried rice, Month Submitted: 2012-05, Score: 5.0
Name: roasted eggplant with harissa potatoes and tomatoes, Month Submitted: 2012-05, Score: 5.0
Name: chihuahua chili, Month Submitted: 2012-05, Score: 5.0
Name: falafels  from ten dollar dinners, Month Submitted: 2012-05, Score: 5.0
Name: fish fingers with fresh mango salsa, Month Submitted: 2012-05, Score: 5.0
Name: kratzede   pancake scrapings, Month Submitted: 2012-05, Score: 5.0
Name: asparagus sauteed in butter and mustard, Month Submitted: 2012-05, Score: 5.0
Name: sushi vinegar, Month Submitted: 2012-05, Score: 5.0
Name: melted onions, Month Submitted: 2012-05, Score: 5.0
Name: mint sweet iced tea, Month Submitted: 2012-05, Score: 5.0
Name: smoked turkey and bean soup, Month Submitted: 2012-05, Score: 5.0
Name: lemon and condensed milk biscuits  botswana, Month Submitted: 2012-05, Score: 5.0
Name: cajun rubbed tilapia, Month Submitt

In [9]:
#Query 2
starting_time = time.time()

query = session.prepare("SELECT name, difficulty, nutrition, steps, description, score FROM recipe.recipes WHERE name = ? LIMIT 1;")

rows = session.execute(query, ['chick greek salad'])
for row in rows:
    print(f"Name: {row.name},\nDifficulty: {row.difficulty},\nNutrition: {row.nutrition},\nSteps: {row.steps},\nDescription: {row.description},\nScore: {row.score}");

rows = session.execute(query, ['banana pineapple freeze'])
for row in rows:
    continue; rows = session.execute(query, ['swedish beet salad'])
for row in rows:
    continue;
rows = session.execute(query, ['authentic bolillos  mexican bread rolls for tortas'])
for row in rows:
    continue;
rows = session.execute(query, ['pretzels   hard and soft'])
for row in rows:
    continue;
rows = session.execute(query, ['tasty tomato ramen'])
for row in rows:
    continue;
rows = session.execute(query, ['chocolate caramel cheesecake'])
for row in rows:
    continue;
rows = session.execute(query, ['banana berry smoothie  no added sugar  sugarless'])
for row in rows:
    continue;
rows = session.execute(query, ['vegetable couscous'])
for row in rows:
    continue;
rows = session.execute(query, ['vegan pastry cream'])
for row in rows:
    continue;

elapsed_time = (time.time() - starting_time) / 10
mlsec = repr(elapsed_time).split('.')[1][:6]
print('Mean Execution time:', time.strftime("%H:%M:%S.{}".format(mlsec), time.gmtime(elapsed_time)))

Mean Execution time: 00:00:00.003149


In [10]:
#Query 3
starting_time = time.time()

query = session.prepare("SELECT name, difficulty, score FROM recipe.recipes_by_difficulty WHERE difficulty=?",)

rows = session.execute(query, ['easy'])
i = 0;
for row in rows:
    if i < 20:
        print(f"Name: {row.name}, Difficulty: {row.difficulty}, Score: {row.score}");
    i+=1

rows = session.execute(query, ['easy'])
for row in rows:
    continue;
rows = session.execute(query, ['medium'])
for row in rows:
    continue;
rows = session.execute(query, ['hard'])
for row in rows:
    continue;
rows = session.execute(query, ['very_hard'])
for row in rows:
    continue;
rows = session.execute(query, ['easy'])
for row in rows:
    continue;
rows = session.execute(query, ['medium'])
for row in rows:
    continue;
rows = session.execute(query, ['hard'])
for row in rows:
    continue;
rows = session.execute(query, ['very_hard'])
for row in rows:
    continue;
rows = session.execute(query, ['easy'])
for row in rows:
    continue;
    
elapsed_time = (time.time() - starting_time) / 10
mlsec = repr(elapsed_time).split('.')[1][:6]
print('Execution time:', time.strftime("%H:%M:%S.{}".format(mlsec), time.gmtime(elapsed_time)))

Name: a jad   cucumber pickle, Difficulty: easy, Score: 5.0
Name: cafe cappuccino, Difficulty: easy, Score: 5.0
Name: caramel apple milkshakes, Difficulty: easy, Score: 5.0
Name: cherry sandwich maker snack, Difficulty: easy, Score: 5.0
Name: cinnamon curry rice, Difficulty: easy, Score: 5.0
Name: christmas snow punch, Difficulty: easy, Score: 5.0
Name: thai fried bananas, Difficulty: easy, Score: 5.0
Name: homemade butter ii, Difficulty: easy, Score: 5.0
Name: mom s first place turkey fruit salad, Difficulty: easy, Score: 5.0
Name: yellow squash frittata, Difficulty: easy, Score: 5.0
Name: iced mochaccino smoothie, Difficulty: easy, Score: 5.0
Name: coffee punch with ice cream floats, Difficulty: easy, Score: 5.0
Name: alaskan blueberry pie, Difficulty: easy, Score: 5.0
Name: whipped cappuccino, Difficulty: easy, Score: 5.0
Name: green mango salad with cilantro vinaigrette, Difficulty: easy, Score: 5.0
Name: double cabbage salad, Difficulty: easy, Score: 5.0
Name: lemon whipped cream,

In [11]:
#Query 4
starting_time = time.time()

query = session.prepare("SELECT name, tag, date_submitted FROM recipe.recipes_by_tag WHERE tag = ? ORDER BY date_submitted;")

rows = session.execute(query, ['crock-pot-slow-cooker'])
i = 0;
for row in rows:
    if i < 20:
        print(f"Name: {row.name}, Tag: {row.tag}, Date Submitted: {row.date_submitted}");
    i+=1

rows = session.execute(query, ['malaysian'])
for row in rows:
    continue;
rows = session.execute(query, ['spanish'])
for row in rows:
    continue;
rows = session.execute(query, ['shakes'])
for row in rows:
    continue;
rows = session.execute(query, ['mixer'])
for row in rows:
    continue;
rows = session.execute(query, ['60-minutes-or-less'])
for row in rows:
    continue;
rows = session.execute(query, ['dietary'])
for row in rows:
    continue;
rows = session.execute(query, ['chocolate'])
for row in rows:
    continue;
rows = session.execute(query, ['for-large-groups'])
for row in rows:
    continue;
rows = session.execute(query, ['taste-mood'])
for row in rows:
    continue;

elapsed_time = (time.time() - starting_time) / 10
mlsec = repr(elapsed_time).split('.')[1][:6]
print('Execution time:', time.strftime("%H:%M:%S.{}".format(mlsec), time.gmtime(elapsed_time)))

Name: crock pot curried carrots, Tag: crock-pot-slow-cooker, Date Submitted: 1999-08-07
Name: crock pot chocolate zucchini nut bread, Tag: crock-pot-slow-cooker, Date Submitted: 1999-08-10
Name: all purpose crock pot chicken, Tag: crock-pot-slow-cooker, Date Submitted: 1999-08-10
Name: barbecued pork strips, Tag: crock-pot-slow-cooker, Date Submitted: 1999-08-11
Name: slow cooker bbq pinto beans, Tag: crock-pot-slow-cooker, Date Submitted: 1999-08-14
Name: crock pot potatoes, Tag: crock-pot-slow-cooker, Date Submitted: 1999-08-14
Name: orange chicken  crock pot, Tag: crock-pot-slow-cooker, Date Submitted: 1999-08-26
Name: ground beef stuffed green bell peppers ii   oven or crock pot, Tag: crock-pot-slow-cooker, Date Submitted: 1999-09-23
Name: creole black beans and rice, Tag: crock-pot-slow-cooker, Date Submitted: 1999-09-30
Name: crock pot stuffed chicken rolls, Tag: crock-pot-slow-cooker, Date Submitted: 1999-10-11
Name: slow cooker chicken stew, Tag: crock-pot-slow-cooker, Date Sub

In [12]:
#Query 5
starting_time = time.time()

query = session.prepare("SELECT name, tag, score FROM recipe.recipes_by_tag WHERE tag = ? LIMIT 20;")

rows = session.execute(query, ['cocktail'])
i = 0;
for row in rows:
    if i < 20:
        print(f"Name: {row.name}, Tag: {row.tag}, Score: {row.score}");
    i+=1;
    
rows = session.execute(query, ['to-go'])
for row in rows:
    continue;
rows = session.execute(query, ['kid-friendly'])
for row in rows:
    continue;
rows = session.execute(query, ['dietary'])
for row in rows:
    continue;
rows = session.execute(query, ['desserts'])
for row in rows:
    continue;
rows = session.execute(query, ['occasion'])
for row in rows:
    continue;
rows = session.execute(query, ['seasonal'])
for row in rows:
    continue;
rows = session.execute(query, ['grilling'])
for row in rows:
    continue;
rows = session.execute(query, ['brunch'])
for row in rows:
    continue;
rows = session.execute(query, ['beginner-cook'])
for row in rows:
    continue;

elapsed_time = (time.time() - starting_time) / 10
mlsec = repr(elapsed_time).split('.')[1][:6]
print('Execution time:', time.strftime("%H:%M:%S.{}".format(mlsec), time.gmtime(elapsed_time)))

Execution time: 00:00:00.002620


In [13]:
cluster.shutdown()